In [1]:
from keras import Sequential
import numpy as np
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.layers import Flatten
from keras import layers
from sklearn.datasets import fetch_20newsgroups

# **3. Apply the code on 20_newsgroup data set**

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True)
sentences = newsgroups_train.data
y = newsgroups_train.target
print(np.unique(y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [3]:
# tokenizing data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences)

In [4]:
# getting the vocabulary of data
max_review_len= max([len(s.split()) for s in sentences])
vocab_size= len(tokenizer.word_index)+1
sentences = tokenizer.texts_to_sequences(sentences)
padded_docs= pad_sequences(sentences,maxlen=max_review_len)
print(padded_docs.shape)

(11314, 11821)


In [5]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.25, random_state=1000)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_review_len))
model.add(Flatten())
model.add(layers.Dense(300, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
history=model.fit(X_train,y_train, epochs=2, verbose=True, validation_data=(X_test,y_test), batch_size=512)

Epoch 1/2
17/17 [==============================] - 18s 896ms/step - loss: 15.3207 - acc: 0.0515 - val_loss: 7.1511 - val_acc: 0.0463
Epoch 2/2
17/17 [==============================] - 15s 876ms/step - loss: 3.9559 - acc: 0.0596 - val_loss: 3.1540 - val_acc: 0.0612


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11821, 50)         6707150   
                                                                 
 flatten (Flatten)           (None, 591050)            0         
                                                                 
 dense (Dense)               (None, 300)               177315300 
                                                                 
 dense_1 (Dense)             (None, 20)                6020      
                                                                 
Total params: 184,028,470
Trainable params: 184,028,470
Non-trainable params: 0
_________________________________________________________________


In [8]:
[test_loss, test_acc] = model.evaluate(X_test,y_test)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

89/89 [==============================] - 2s 21ms/step - loss: 3.1540 - acc: 0.0612
Evaluation result on Test Data : Loss = 3.1539864540100098, accuracy = 0.06115235015749931
